# 🩺 **Sistem Rekomendasi Artikel Berdasarkan Kondisi Kesehatan Menggunakan MLP**

## **Deskripsi Proyek**

Proyek ini bertujuan membangun sistem rekomendasi artikel kesehatan berbasis kondisi pengguna. Sistem memanfaatkan data medis seperti tekanan darah, gula darah, suhu tubuh, trimester kehamilan, dan gejala-gejala umum (misalnya pusing, diare, sulit tidur), kemudian memprediksi kondisi kesehatan pengguna dan merekomendasikan artikel yang relevan berdasarkan tag.


### **Imports**

In [1]:
import pandas as pd
import numpy as np
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.metrics import f1_score, precision_score, recall_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import matplotlib.pyplot as plt

## **Dataset**

### **Dataset Artikel**

In [2]:
# from google.colab import files
# uploaded = files.upload()

In [3]:
df_artikel_new = pd.read_csv("data_artikel_new.csv")
df_artikel_new.head()

,id,judul,kategori,deskripsi,content,tag
0,1,Mengatasi Rasa Lelah yang Tak Tertahankan,Kesehatan Ibu Hamil,Strategi mengelola fatigue ekstrim selama keha...,<p>Kehamilan adalah masa penuh harapan dan kea...,bradikardia|gula_darah_rendah
1,2,Demam Berkepanjangan - Kapan Harus Ke Dokter?,Kesehatan Ibu Hamil,Membedakan demam biasa dengan infeksi serius.,<p>Demam selama kehamilan bisa menjadi hal yan...,demam_lebih_2_hari
2,3,Diare Tak Kunjung Berhenti - Solusi Aman,Gangguan Pencernaan,Penanganan diare kronis tanpa membahayakan janin.,<p>Diare adalah salah satu keluhan pencernaan ...,diare_berulang
3,4,Ngidam Manis di Malam Hari - Amankah?,Nutrisi Ibu Hamil,Mengelola keinginan makan manis secara sehat.,<p>Ngidam makanan manis di malam hari merupaka...,gula_darah_tinggi
4,5,Keringat Dingin dan Gemetar Tiba-tiba,Kesehatan Ibu Hamil,Mengenali gejala hipoglikemia darurat.,"<p>Selama kehamilan, tubuh mengalami berbagai ...",gula_darah_rendah


Berisi metadata artikel:
- `judul`, `kategori`, `deskripsi`, dan `tag`
- Kolom `tag` mendeskripsikan topik artikel, bisa lebih dari satu, dipisahkan dengan `|` (multi-label juga).

In [4]:
df_artikel_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         80 non-null     int64 
 1   judul      80 non-null     object
 2   kategori   80 non-null     object
 3   deskripsi  80 non-null     object
 4   content    80 non-null     object
 5   tag        80 non-null     object
dtypes: int64(1), object(5)
memory usage: 3.9+ KB


In [5]:
# Menampilkan nilai unik beserta jumlah kemunculannya
df_artikel_new['tag'].value_counts()

tag
sulit_tidur                        13
diare_berulang                      7
hipertensi_ringan                   5
gula_darah_rendah                   5
pusing                              5
demam_lebih_2_hari                  5
pusing|1                            4
pusing|gula_darah_rendah            2
bradikardia                         2
nyeri_perut_berat|2|3               2
diare_berulang|3                    2
hipertensi_berat|3                  2
takikardia                          2
hipertensi_ringan|3                 2
gula_darah_tinggi|2|3               2
gula_darah_tinggi                   2
gula_darah_tinggi|1                 1
bradikardia|3                       1
hipertensi_ringan|2|3               1
gula_darah_rendah|1                 1
nyeri_perut_berat|2                 1
diare_berulang|1                    1
takikardia|3                        1
bradikardia|gula_darah_rendah       1
gula_darah_tinggi|3                 1
nyeri_perut_berat|3                 1
sulit_ti

### **Dataset Pengguna**

Berisi data medis buatan sebanyak 20.000 baris. Fitur:

- **Numerik**:
  - `tekanan_sistolik`, `tekanan_diastolik`, `gula_darah`, `suhu_tubuh`, `trimester`, `denyut_jantung`
- **Kategorikal (Yes/No)**:
  - `demam_lebih_2_hari`, `pusing`, `sulit_tidur`, `nyeri_perut_berat`, `diare_berulang`
- **Target multi-label**:
  - `kategori_kondisi` (misal: `hipertensi_berat|pusing`)

In [6]:
# Daftar gejala tambahan (yes/no)
gejala_list = [
    "demam_lebih_2_hari",
    "pusing",
    "sulit_tidur",
    "nyeri_perut_berat",
    "diare_berulang"
]

# Generate 100 data dummy
data_user = []

for _ in range(20000):
    sistolik = random.randint(90, 170)
    diastolik = random.randint(60, 110)
    gula = random.randint(60, 120)
    suhu = round(random.uniform(36.4, 37.6), 1)
    trimester = random.randint(1, 3)
    denyut_jantung = random.randint(50, 130)

    # Buat jawaban acak untuk gejala (yes/no)
    gejala = {g: random.choice(["yes", "no"]) for g in gejala_list}

    # Tentukan label kondisi
    kondisi = []

    # Kondisi berdasarkan tekanan darah
    if sistolik >= 160 or diastolik >= 100:
        kondisi.append("hipertensi_berat")
    elif sistolik >= 140 or diastolik >= 90:
        kondisi.append("hipertensi_ringan")
    elif sistolik >= 120 or diastolik >= 80:
        kondisi.append("prehipertensi")

    # Kondisi berdasarkan gula darah
    if gula >= 110:
        kondisi.append("gula_darah_tinggi")
    elif gula <= 75:
        kondisi.append("gula_darah_rendah")

    # Kondisi berdasarkan denyut jantung
    if denyut_jantung < 60:
        kondisi.append("bradikardia")
    elif denyut_jantung > 100:
        kondisi.append("takikardia")

    # Kondisi berdasarkan gejala
    for g in gejala_list:
        if gejala[g] == "yes":
            kondisi.append(g)

    if not kondisi:
        kondisi.append("normal")

    # Gabungkan semua data ke dalam satu baris
    row = {
        "tekanan_sistolik": sistolik,
        "tekanan_diastolik": diastolik,
        "gula_darah": gula,
        "suhu_tubuh": suhu,
        "trimester": trimester,
        "denyut_jantung": denyut_jantung,
    }
    row.update(gejala)
    row["kategori_kondisi"] = "|".join(kondisi)  # multi-label disatukan pakai pipe
    data_user.append(row)

# Buat DataFrame
df_user = pd.DataFrame(data_user)

In [7]:
df_user.head(10)

,tekanan_sistolik,tekanan_diastolik,gula_darah,suhu_tubuh,trimester,denyut_jantung,demam_lebih_2_hari,pusing,sulit_tidur,nyeri_perut_berat,diare_berulang,kategori_kondisi
0,108,93,97,36.5,1,89,yes,yes,no,no,yes,hipertensi_ringan|demam_lebih_2_hari|pusing|di...
1,121,102,62,37.6,1,85,yes,yes,no,yes,yes,hipertensi_berat|gula_darah_rendah|demam_lebih...
2,136,109,66,36.8,1,120,yes,yes,yes,no,yes,hipertensi_berat|gula_darah_rendah|takikardia|...
3,150,70,94,36.8,3,62,yes,no,no,yes,yes,hipertensi_ringan|demam_lebih_2_hari|nyeri_per...
4,161,70,120,37.0,3,50,no,yes,no,yes,yes,hipertensi_berat|gula_darah_tinggi|bradikardia...
5,99,102,83,37.2,2,112,yes,yes,no,no,yes,hipertensi_berat|takikardia|demam_lebih_2_hari...
6,169,81,78,37.3,1,95,no,yes,yes,yes,yes,hipertensi_berat|pusing|sulit_tidur|nyeri_peru...
7,123,81,72,36.5,1,52,yes,no,no,yes,no,prehipertensi|gula_darah_rendah|bradikardia|de...
8,147,99,79,37.2,1,108,yes,no,no,no,no,hipertensi_ringan|takikardia|demam_lebih_2_hari
9,96,103,90,36.5,3,123,yes,yes,yes,no,yes,hipertensi_berat|takikardia|demam_lebih_2_hari...


In [8]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tekanan_sistolik    20000 non-null  int64  
 1   tekanan_diastolik   20000 non-null  int64  
 2   gula_darah          20000 non-null  int64  
 3   suhu_tubuh          20000 non-null  float64
 4   trimester           20000 non-null  int64  
 5   denyut_jantung      20000 non-null  int64  
 6   demam_lebih_2_hari  20000 non-null  object 
 7   pusing              20000 non-null  object 
 8   sulit_tidur         20000 non-null  object 
 9   nyeri_perut_berat   20000 non-null  object 
 10  diare_berulang      20000 non-null  object 
 11  kategori_kondisi    20000 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 1.8+ MB


## **Data Preprocessing**

**Input Fitur** (`X`)
- Fitur numerik: distandarisasi dengan `StandardScaler`.
- Fitur gejala: dikonversi ke binary (`yes` → 1, `no` → 0) dengan `pd.get_dummies`.

In [9]:
gejala_list = ["demam_lebih_2_hari", "pusing", "sulit_tidur", "nyeri_perut_berat", "diare_berulang"]
numerik_cols = ["tekanan_sistolik", "tekanan_diastolik", "gula_darah", "suhu_tubuh", "trimester", "denyut_jantung"]

X_num = df_user[numerik_cols].values
X_gejala = df_user[gejala_list].replace({"yes": 1, "no": 0}).values
X_all = np.concatenate([X_num, X_gejala], axis=1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_all)

C:\Users\hp\AppData\Local\Temp\ipykernel_13536\543204671.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_gejala = df_user[gejala_list].replace({"yes": 1, "no": 0}).values


**Target** (`y`)

Label multi-kondisi dari `kategori_kondisi` dipecah dan di-binarisasi menggunakan `MultiLabelBinarizer`.

In [10]:
# Encode labels
y_raw = df_user["kategori_kondisi"].str.split("|")
mlb = MultiLabelBinarizer()
y_encoded = mlb.fit_transform(y_raw)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

## **🧠 Modeling**

Model menggunakan arsitektur Sequential dari Keras:

In [11]:
# MLP model
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(mlb.classes_), activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy')
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
450/450 [==============================] - 5s 4ms/step - loss: 0.2170 - val_loss: 0.1217
Epoch 2/10
450/450 [==============================] - 5s 4ms/step - loss: 0.2170 - val_loss: 0.1217
Epoch 2/10
450/450 [==============================] - 2s 4ms/step - loss: 0.0998 - val_loss: 0.0880
Epoch 3/10
450/450 [==============================] - 2s 4ms/step - loss: 0.0998 - val_loss: 0.0880
Epoch 3/10
450/450 [==============================] - 2s 4ms/step - loss: 0.0783 - val_loss: 0.0742
Epoch 4/10
450/450 [==============================] - 2s 4ms/step - loss: 0.0783 - val_loss: 0.0742
Epoch 4/10
450/450 [==============================] - 2s 4ms/step - loss: 0.0659 - val_loss: 0.0617
Epoch 5/10
450/450 [==============================] - 2s 4ms/step - loss: 0.0659 - val_loss: 0.0617
Epoch 5/10
450/450 [==============================] - 2s 4ms/step - loss: 0.0539 - val_loss: 0.0486
Epoch 6/10
450/450 [==============================] - 2s 4ms/step - loss: 0.0539 - val_loss: 0.0486


In [14]:
# Evaluasi
y_pred = model.predict(X_test)
y_pred_bin = (y_pred > 0.3).astype(int)

print("F1 Score:", f1_score(y_test, y_pred_bin, average="macro"))
print("Precision:", precision_score(y_test, y_pred_bin, average="macro"))
print("Recall:", recall_score(y_test, y_pred_bin, average="macro"))

125/125 [==============================] - 0s 2ms/step
F1 Score: 0.954498109958425
F1 Score: 0.954498109958425
Precision: 0.9386205377392816
Recall: 0.9738941385447879
Precision: 0.9386205377392816
Recall: 0.9738941385447879


## **Inference dan Rekomendasi**

In [13]:
# Pilih user
sample_index = 0
X_user = X_test[sample_index].reshape(1, -1)

# Prediksi Kondisi
predicted_probs = model.predict(X_user)[0]

# Top-N Kondisi
top_n = 5
top_indices = np.argsort(predicted_probs)[::-1][:top_n]
top_kondisi = mlb.classes_[top_indices]

# Filter artikel berdasarkan tag
def get_top_articles(tags, df_artikel, n=5):
    return df_artikel[df_artikel['tag'].apply(
        lambda x: any(tag in x.split('|') for tag in tags))].head(n)[['judul', 'deskripsi', 'tag']]

top_articles = get_top_articles(top_kondisi, df_artikel_new)
print("Rekomendasi artikel berdasarkan kondisi:", top_kondisi)
top_articles


1/1 [==============================] - 0s 118ms/step
Rekomendasi artikel berdasarkan kondisi: ['takikardia' 'hipertensi_berat' 'sulit_tidur' 'pusing' 'diare_berulang']
Rekomendasi artikel berdasarkan kondisi: ['takikardia' 'hipertensi_berat' 'sulit_tidur' 'pusing' 'diare_berulang']


,judul,deskripsi,tag
2,Diare Tak Kunjung Berhenti - Solusi Aman,Penanganan diare kronis tanpa membahayakan janin.,diare_berulang
7,Pandangan Berkabut Disertai Sakit Kepala,Gejala visual yang perlu evaluasi segera.,hipertensi_berat|pusing
8,Sesak Napas Saat Aktivitas Ringan,Membedakan sesak normal dengan gejala serius.,takikardia
9,Bengkak Kaki Parah di Sore Hari,Membedakan edema normal dengan tanda bahaya.,hipertensi_berat|3
10,Detak Jantung Terasa Sampai ke Leher,Memahami palpitasi selama kehamilan.,takikardia|2


In [16]:
# Simpan model ke file HDF5
model.save('model_rekomendasi_artikel', save_format='tf')

INFO:tensorflow:Assets written to: model_rekomendasi_artikel\assets
